In [8]:
import os
import dagshub
import mlflow

In [2]:
%pwd

'd:\\Projects\\Chest-Disease-Classification-from-Chest-CT-Scan-Image\\research'

In [6]:
os.chdir("../")

In [9]:

dagshub.init(repo_owner='utkarsh.shelke03', repo_name='Chest-Disease-Classification-from-Chest-CT-Scan-Image', mlflow=True)

with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

Accessing as utkarsh.shelke03

Initialized MLflow to track repo "utkarsh.shelke03/Chest-Disease-Classification-from-Chest-CT-Scan-Image"

Repository utkarsh.shelke03/Chest-Disease-Classification-from-Chest-CT-Scan-Image initialized!

2024/10/29 21:24:35 INFO mlflow.tracking._tracking_service.client: 🏃 View run bouncy-shrimp-928 at: https://dagshub.com/utkarsh.shelke03/Chest-Disease-Classification-from-Chest-CT-Scan-Image.mlflow/#/experiments/0/runs/4d42a4bd6fe747ecb0f5816db3f27bdf.
2024/10/29 21:24:35 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/utkarsh.shelke03/Chest-Disease-Classification-from-Chest-CT-Scan-Image.mlflow/#/experiments/0.


In [10]:
remote_server_uri = "https://dagshub.com/utkarsh.shelke03/Chest-Disease-Classification-from-Chest-CT-Scan-Image.mlflow"

In [11]:
import tensorflow as tf

In [13]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model : Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [14]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [15]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/CHest-CT-Scan-data",
            mlflow_uri="https://dagshub.com/utkarsh.shelke03/Chest-Disease-Classification-from-Chest-CT-Scan-Image.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [16]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [24]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
        
    def _valid_generator(self):
        
        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.30
        )
    
        dataflow_kwargs = dict(
        target_size= self.config.params_image_size[:-1],
        batch_size= self.config.params_batch_size,
        interpolation="bilinear"
        )    
    
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
        **datagenerator_kwargs
        )
    
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset='validation',
            shuffle=False,
            **dataflow_kwargs
    )
    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()
        
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"),data=scores)
        
    def log_into_mlflow(self): 
        mlflow.set_registry_uri(self.config.mlflow_uri) 
        tracking_url_type_store= urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.lof_metrics(
                {"loss":self.score[0], "accuracy":self.score[1]}
            )
            # Model registry does not work with file store
            
            if tracking_url_type_store != "file":
                # Register the model
                # There are other ways to use the model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/models.html#registering-models-with-the-model-registry
                mlflow.keras.log_model(self.model, "model", registered_model_name ="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [26]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()
    
except Exception as e:
    raise e

[2024-10-29 22:03:15,190: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-10-29 22:03:15,193: INFO: common: yaml file: params.yaml loaded successfully]
[2024-10-29 22:03:15,194: INFO: common: Directory created: artifacts]


ValueError: Invalid class_mode: bilinear; expected one of: {'input', None, 'categorical', 'binary', 'sparse'}